In [6]:
import nfl_data_py as nfl
from scoring import ScoringFormat, StandardScoringFormat

In [ ]:
format = StandardScoringFormat()

In [ ]:
format.r

In [7]:
test_data = nfl.import_weekly_data([2024])
test_data.head(5)

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,8.580000,8.580000
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,15.140000,15.140000
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,3,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.040001,21.040001
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,4,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.600000,11.600000
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,5,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.760000,11.760000


In [8]:
arod_data = test_data.query('player_display_name == "Aaron Rodgers"')

In [9]:
# Dict of {column_name: attr_name} for use with the nfl-data-py library
stat_mapping_nfl_py = {
    #PASSING
    'passing_yards' : 'pass_yards_value',
    'passing_tds' : 'pass_tds_value',
    'interceptions' : 'pass_ints_value',
    #RUSHING
    'rushing_yards' : 'rush_yards_value',
    'rushing_tds': 'rush_tds_value',
    #RECEIVING
    'receptions' : 'receptions_value',
    'receiving_yards' : 'rec_yards_value',
    'receiving_tds' : 'rec_tds_value',
    #MISC
    'sack_fumbles_lost' : 'fumble_lost_value', # This data source distinguishes between different kinds of fumbles
    'rushing_fumbles_lost' : 'fumble_lost_value',
    'receiving_fumbles_lost' : 'fumble_lost_value',
    'passing_2pt_conversions' : 'two_point_conversions_value', # Same for 2pt conversions
    'rushing_2pt_conversions' : 'two_point_conversions_value',
    'receiving_2pt_conversions' : 'two_point_conversions_value',    
}

In [10]:
type(arod_data.loc[0])

pandas.core.series.Series

In [11]:
def calculate_fantasy_points(
    stats_row: pd.Series, 
    scoring_format: ScoringFormat, 
    stat_mapping:dict, 
    debug=False) -> float:
    """Calculates the total points scored by one row of stats. """
    total_points = 0.0
    for column, scoring_attribute in stat_mapping.items():
        if column in stats_row and hasattr(scoring_format, scoring_attribute):
            total_points += stats_row[column] * getattr(scoring_format, scoring_attribute)
            if debug == True:
                print(f"{column} : {stats_row[column]}, {scoring_attribute} : {getattr(scoring_format, scoring_attribute)}")
    return total_points
    

In [12]:
def calculate_fantasy_points_by_category(
    stats_row: pd.Series, 
    scoring_format: ScoringFormat, 
    stat_mapping:dict, 
    debug=False) -> dict:
    """Calculates the total points scored by one row of stats, returning a dict mapping each scoring category to its respective points. """
    points_by_category = {}
    for column, scoring_attribute in stat_mapping.items():
        if column in stats_row and hasattr(scoring_format, scoring_attribute):
            points_by_category[scoring_attribute] =  stats_row[column] * getattr(scoring_format, scoring_attribute)
            if debug == True:
                print(f"{column} : {stats_row[column]}, {scoring_attribute} : {getattr(scoring_format, scoring_attribute)}")
    return points_by_category

In [13]:
calculate_fantasy_points_by_category(arod_data.loc[0], StandardScoringFormat(), stat_mapping_nfl_py, debug=False)

{'pass_yards_value': 6.68,
 'pass_tds_value': 4,
 'pass_ints_value': -2.0,
 'rush_yards_value': -0.1,
 'rush_tds_value': 0,
 'receptions_value': 0,
 'rec_yards_value': 0.0,
 'rec_tds_value': 0,
 'fumble_lost_value': -0.0,
 'two_point_conversions_value': 0}

In [14]:
calculate_fantasy_points(arod_data.loc[0], StandardScoringFormat(), stat_mapping_nfl_py, debug=False)

8.58

In [15]:
arod_data.loc[0]['rushing_yards']

-1.0

In [16]:
arod_data['calc_fantasy_points'] = arod_data.apply(
    lambda row: calculate_fantasy_points(row, StandardScoringFormat(), stat_mapping_nfl_py), axis=1
)


/var/folders/9_/yz683g6908g59bpvdwvzm7dw0000gn/T/ipykernel_28186/2755742469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arod_data['calc_fantasy_points'] = arod_data.apply(


In [17]:
arod_data

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,calc_fantasy_points
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,1,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,8.580000,8.580000,8.58
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,15.140000,15.140000,15.14
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,3,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,21.040001,21.040001,21.04
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,4,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,11.600000,11.600000,11.60
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,5,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,11.760000,11.760000,11.76
5,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,6,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,17.760000,17.760000,17.76
6,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,7,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,13.040000,13.040000,13.04
7,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,8,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,17.320000,17.320000,17.32
8,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,9,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,20.340000,20.340000,20.34
9,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,10,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,4.040000,4.040000,4.04
